#### Sunday, December 22, 2024

Whelp, I am now thinking the reason function calling is failing is because LMStudio does NOT yet support function calling! I am currently using LMStudio 0.3.5 (Build 1). The current [LMStudio Web Site](https://lmstudio.ai/) says 'Function Calling is now in Beta! Try out the [latest build](https://lmstudio.ai/beta-releases)' which as of today is Version 0.3.5 Build 8, last updated on 2024-11-27, so I am gonna download this and give it a go.

LMStudio has docs about Tool User [here](https://lmstudio.ai/docs/advanced/tool-use).

Hmm so I did download this version, and when I spin it up, it shows 'Build 9'. 

Nice! Running the code using 'mistral-nemo-instruct-2407', and now I AM getting back the results I want! Great!

///////////////////////////////

OK! This notebook was copied from 'function_calliing_lmstudio.ipynb' and will be used to test calling a model served up from Ollama. 

I am running Ollama version 0.5.4 and have run the commana 'ollama run llama3-groq-tool-use'.

#### Saturday, December 21, 2024

I began this notebook using 'lmstudio-community/Qwen2.5-14B-Instruct-GGUF' but this does not appear to support function calling.
OK. Since I so far have not been able to get any local model to perform function calling, I just ran stuff against OpenAI because I wanted to see what DOES work, so then I know how a local model is supposed to work.

To see which models on HuggingFace support function calling, go [here](https://huggingface.co/models?other=function-calling)

DO NOT RUN THIS AGAIN! I WANT TO KEEP THE OUTPUT FOR FUTURE REFERENCE!

OK! So this notebook was copied from 'function_calling.ipynb' and will be used to try and find some local model that supports function calling. 

In [1]:
# Deliberately set the OPENAI_API_KEY to an invalid value to ensure that the code is not using it.
import os
os.environ['OPENAI_API_KEY'] = "Nope!"

## OpenAI Function Calling with Tool Usage

In [2]:
from openai import OpenAI
import json
from os import getenv

# client = OpenAI(api_key=getenv("OPENAI_API_KEY"))

# 4090 ... 

# api_key = "LMStudio"
api_key = "Ollama"
# lmstudio = "http://192.168.2.16:1234/v1"
# lmstudio = "http://127.0.0.1:1234/v1"
ollama = "http://localhost:11434/v1/"
# model = "qwen2.5-14b-instruct@q8_0" # lmstudio-community/Qwen2.5-14B-Instruct-GGUF :  Qwen2.5-14B-Instruct-Q8_0.gguf
# model = "qwen2.5-14b-instruct@q4_k_m" # lmstudio-community/Qwen2.5-14B-Instruct-GGUF :  Qwen2.5-14B-Instruct-Q4_K_M.gguf
# model = "lmstudio-community/mistral-nemo-instruct-2407"   # lmstudio-community/Mistral-Nemo-Instruct-2407-GGUF : Mistral-Nemo-Instruct-2407-Q4_K_M.gguf

# model = "lmstudio-community/Llama-3-Groq-8B-Tool-Use-GGUF"

# model = "lmstudio-community/Mistral-Nemo-Instruct-2407-GGUF"

model = "llama3-groq-tool-use"

# chat = ChatOpenAI(base_url=lmstudio, model=model, api_key="LMStudio",  model_kwargs={"response_format": {"type": "json_object"}})
client = OpenAI(base_url=ollama, api_key=api_key)

I grabbed this code in the next cell from the ollama docs found [here](https://github.com/ollama/ollama/blob/main/docs/openai.md)

For this cell to run without errors, you need to have already downloaded to Ollama the models 'llama3.2','llava' and 'all-minilm'. 

The fact that you can switch from one ollama model to another at run time is pretty impressive! Can LMStudio do this??

In [11]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',

    # required but ignored
    api_key='ollama',
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': 'Say this is a test',
        }
    ],
    model='llama3.2',
)

response = client.chat.completions.create(
    model="llava",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAG0AAABmCAYAAADBPx+VAAAACXBIWXMAAAsTAAALEwEAmpwYAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAA3VSURBVHgB7Z27r0zdG8fX743i1bi1ikMoFMQloXRpKFFIqI7LH4BEQ+NWIkjQuSWCRIEoULk0gsK1kCBI0IhrQVT7tz/7zZo888yz1r7MnDl7z5xvsjkzs2fP3uu71nNfa7lkAsm7d++Sffv2JbNmzUqcc8m0adOSzZs3Z+/XES4ZckAWJEGWPiCxjsQNLWmQsWjRIpMseaxcuTKpG/7HP27I8P79e7dq1ars/yL4/v27S0ejqwv+cUOGEGGpKHR37tzJCEpHV9tnT58+dXXCJDdECBE2Ojrqjh071hpNECjx4cMHVycM1Uhbv359B2F79+51586daxN/+pyRkRFXKyRDAqxEp4yMlDDzXG1NPnnyJKkThoK0VFd1ELZu3TrzXKxKfW7dMBQ6bcuWLW2v0VlHjx41z717927ba22U9APcw7Nnz1oGEPeL3m3p2mTAYYnFmMOMXybPPXv2bNIPpFZr1NHn4HMw0KRBjg9NuRw95s8PEcz/6DZELQd/09C9QGq5RsmSRybqkwHGjh07OsJSsYYm3ijPpyHzoiacg35MLdDSIS/O1yM778jOTwYUkKNHWUzUWaOsylE00MyI0fcnOwIdjvtNdW/HZwNLGg+sR1kMepSNJXmIwxBZiG8tDTpEZzKg0GItNsosY8USkxDhD0Rinuiko2gfL/RbiD2LZAjU9zKQJj8RDR0vJBR1/Phx9+PHj9Z7REF4nTZkxzX4LCXHrV271qXkBAPGfP/atWvu/PnzHe4C97F48eIsRLZ9+3a3f/9+87dwP1JxaF7/3r17ba+5l4EcaVo0lj3SBq5kGTJSQmLWMjgYNei2GPT1MuMqGTDEFHzeQSP2wi/jGnkmPJ/nhccs44jvDAxpVcxnq0F6eT8h4ni/iIWpR5lPyA6ETkNXoSukvpJAD3AsXLiwpZs49+fPn5ke4j10TqYvegSfn0OnafC+Tv9ooA/JPkgQysqQNBzagXY55nO/oa1F7qvIPWkRL12WRpMWUvpVDYmxAPehxWSe8ZEXL20sadYIozfmNch4QJPAfeJgW3rNsnzphBKNJM2KKODo1rVOMRYik5ETy3ix4qWNI81qAAirizgMIc+yhTytx0JWZuNI03qsrgWlGtwjoS9XwgUhWGyhUaRZZQNNIEwCiXD16tXcAHUs79co0vSD8rrJCIW98pzvxpAWyyo3HYwqS0+H0BjStClcZJT5coMm6D2LOF8TolGJtK9fvyZpyiC5ePFi9nc/oJU4eiEP0jVoAnHa9wyJycITMP78+eMeP37sXrx44d6+fdt6f82aNdkx1pg9e3Zb5W+RSRE+n+VjksQWifvVaTKFhn5O8my63K8Qabdv33b379/PiAP//vuvW7BggZszZ072/+TJk91YgkafPn166zXB1rQHFvouAWHq9z3SEevSUerqCn2/dDCeta2jxYbr69evk4MHDyY7d+7MjhMnTiTPnz9Pfv/+nfQT2ggpO2dMF8cghuoM7Ygj5iWCqRlGFml0QC/ftGmTmzt3rmsaKDsgBSPh0/8yPeLLBihLkOKJc0jp8H8vUzcxIA1k6QJ/c78tWEyj5P3o4u9+jywNPdJi5rAH9x0KHcl4Hg570eQp3+vHXGyrmEeigzQsQsjavXt38ujRo44LQuDDhw+TW7duRS1HGgMxhNXHgflaNTOsHyKvHK5Ijo2jbFjJBQK9YwFd6RVMzfgRBmEfP37suBBm/p49e1qjEP2mwTViNRo0VJWH1deMXcNK08uUjVUu7s/zRaL+oLNxz1bpANco4npUgX4G2eFbpDFyQoQxojBCpEGSytmOH8qrH5Q9vuzD6ofQylkCUmh8DBAr+q8JCyVNtWQIidKQE9wNtLSQnS4jDSsxNHogzFuQBw4cyM61UKVsjfr3ooBkPSqqQHesUPWVtzi9/vQi1T+rJj7WiTz4Pt/l3LxUkr5P2VYZaZ4URpsE+st/dujQoaBBYokbrz/8TJNQYLSonrPS9kUaSkPeZyj1AWSj+d+VBoy1pIWVNed8P0Ll/ee5HdGRhrHhR5GGN0r4LGZBaj8oFDJitBTJzIZgFcmU0Y8ytWMZMzJOaXUSrUs5RxKnrxmbb5YXO9VGUhtpXldhEUogFr3IzIsvlpmdosVcGVGXFWp2oU9kLFL3dEkSz6NHEY1sjSRdIuDFWEhd8KxFqsRi1uM/nz9/zpxnwlESONdg6dKlbsaMGS4EHFHtjFIDHwKOo46l4TxSuxgDzi+rE2jg+BaFruOX4HXa0Nnf1lwAPufZeF8/r6zD97WK2qFnGjBxTw5qNGPxT+5T/r7/7RawFC3j4vTp09koCxkeHjqbHJqArmH5UrFKKksnxrK7FuRIs8STfBZv+luugXZ2pR/pP9Ois4z+TiMzUUkUjD0iEi1fzX8GmXyuxUBRcaUfykV0YZnlJGKQpOiGB76x5GeWkWWJc3mOrK6S7xdND+W5N6XyaRgtWJFe13GkaZnKOsYqGdOVVVbGupsyA/l7emTLHi7vwTdirNEt0qxnzAvBFcnQF16xh/TMpUuXHDowhlA9vQVraQhkudRdzOnK+04ZSP3DUhVSP61YsaLtd/ks7ZgtPcXqPqEafHkdqa84X6aCeL7YWlv6edGFHb+ZFICPlljHhg0bKuk0CSvVznWsotRu433alNdFrqG45ejoaPCaUkWERpLXjzFL2Rpllp7PJU2a/v7Ab8N05/9t27Z16KUqoFGsxnI9EosS2niSYg9SpU6B4JgTrvVW1flt1sT+0ADIJU2maXzcUTraGCRaL1Wp9rUMk16PMom8QhruxzvZIegJjFU7LLCePfS8uaQdPny4jTTL0dbee5mYokQsXTIWNY46kuMbnt8Kmec+LGWtOVIl9cT1rCB0V8WqkjAsRwta93TbwNYoGKsUSChN44lgBNCoHLHzquYKrU6qZ8lolCIN0Rh6cP0Q3U6I6IXILYOQI513hJaSKAorFpuHXJNfVlpRtmYBk1Su1obZr5dnKAO+L10Hrj3WZW+E3qh6IszE37F6EB+68mGpvKm4eb9bFrlzrok7fvr0Kfv727dvWRmdVTJHw0qiiCUSZ6wCK+7XL/AcsgNyL74DQQ730sv78Su7+t/A36MdY0sW5o40ahslXr58aZ5HtZB8GH64m9EmMZ7FpYw4T6QnrZfgenrhFxaSiSGXtPnz57e9TkNZLvTjeqhr734CNtrK41L40sUQckmj1lGKQ0rC37x544r8eNXRpnVE3ZZY7zXo8NomiO0ZUCj2uHz58rbXoZ6gc0uA+F6ZeKS/jhRDUq8MKrTho9fEkihMmhxtBI1DxKFY9XLpVcSkfoi8JGnToZO5sU5aiDQIW716ddt7ZLYtMQlhECdBGXZZMWldY5BHm5xgAroWj4C0hbYkSc/jBmggIrXJWlZM6pSETsEPGqZOndr2uuuR5rF169a2HoHPdurUKZM4CO1WTPqaDaAd+GFGKdIQkxAn9RuEWcTRyN2KSUgiSgF5aWzPTeA/lN5rZubMmR2bE4SIC4nJoltgAV/dVefZm72AtctUCJU2CMJ327hxY9t7EHbkyJFseq+EJSY16RPo3Dkq1kkr7+q0bNmyDuLQcZBEPYmHVdOBiJyIlrRDq41YPWfXOxUysi5fvtyaj+2BpcnsUV/oSoEMOk2CQGlr4ckhBwaetBhjCwH0ZHtJROPJkyc7UjcYLDjmrH7ADTEBXFfOYmB0k9oYBOjJ8b4aOYSe7QkKcYhFlq3QYLQhSidNmtS2RATwy8YOM3EQJsUjKiaWZ+vZToUQgzhkHXudb/PW5YMHD9yZM2faPsMwoc7RciYJXbGuBqJ1UIGKKLv915jsvgtJxCZDubdXr165mzdvtr1Hz5LONA8jrUwKPqsmVesKa49S3Q4WxmRPUEYdTjgiUcfUwLx589ySJUva3oMkP6IYddq6HMS4o55xBJBUeRjzfa4Zdeg56QZ43LhxoyPo7Lf1kNt7oO8wWAbNwaYjIv5lhyS7kRf96dvm5Jah8vfvX3flyhX35cuX6HfzFHOToS1H4BenCaHvO8pr8iDuwoUL7tevX+b5ZdbBair0xkFIlFDlW4ZknEClsp/TzXyAKVOmmHWFVSbDNw1l1+4f90U6IY/q4V27dpnE9bJ+v87QEydjqx/UamVVPRG+mwkNTYN+9tjkwzEx+atCm/X9WvWtDtAb68Wy9LXa1UmvCDDIpPkyOQ5ZwSzJ4jMrvFcr0rSjOUh+GcT4LSg5ugkW1Io0/SCDQBojh0hPlaJdah+tkVYrnTZowP8iq1F1TgMBBauufyB33x1v+NWFYmT5KmppgHC+NkAgbmRkpD3yn9QIseXymoTQFGQmIOKTxiZIWpvAatenVqRVXf2nTrAWMsPnKrMZHz6bJq5jvce6QK8J1cQNgKxlJapMPdZSR64/UivS9NztpkVEdKcrs5alhhWP9NeqlfWopzhZScI6QxseegZRGeg5a8C3Re1Mfl1ScP36ddcUaMuv24iOJtz7sbUjTS4qBvKmstYJoUauiuD3k5qhyr7QdUHMeCgLa1Ear9NquemdXgmum4fvJ6w1lqsuDhNrg1qSpleJK7K3TF0Q2jSd94uSZ60kK1e3qyVpQK6PVWXp2/FC3mp6jBhKKOiY2h3gtUV64TWM6wDETRPLDfSakXmH3w8g9Jlug8ZtTt4kVF0kLUYYmCCtD/DrQ5YhMGbA9L3ucdjh0y8kOHW5gU/VEEmJTcL4Pz/f7mgoAbYkAAAAAElFTkSuQmCC",
                },
            ],
        }
    ],
    max_tokens=300,
)

completion = client.completions.create(
    model="llama3.2",
    prompt="Say this is a test",
)

list_completion = client.models.list()

model = client.models.retrieve("llama3.2")

embeddings = client.embeddings.create(
    model="all-minilm",
    input=["why is the sky blue?", "why is the grass green?"],
)

In [1]:
# Run this if you do actually want to use OpenAI, and of course, do not set the key to something bogus ... 
# from openai import OpenAI
# import json
# from os import getenv
# client = OpenAI(api_key=getenv("OPENAI_API_KEY"))


In [3]:
def schedule_meeting(date, time, attendees):
    # Connect to calendar service:
    return { "event_id": "1234", "status": "Meeting scheduled successfully!",
            "date": date, "time": time, "attendees": attendees }

OPENAI_FUNCTIONS = {
    "schedule_meeting": schedule_meeting
}

In [4]:
# Our predefined function JSON schema:
functions = [
    {
        "type": "function",
        "function": {
            "type": "object",
            "name": "schedule_meeting",
            "description": "Set a meeting at a specified date and time for designated attendees",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "format": "date"},
                    "time": {"type": "string", "format": "time"},
                    "attendees": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["date", "time", "attendees"],
            },
        },
    }
]

In [5]:
# Start the conversation:
messages = [
    {
        "role": "user",
        "content": "Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob.",
    }
]

In [6]:
# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    # model="gpt-3.5-turbo-1106",
    # model="gpt-3.5-turbo",
    model = model ,
    messages=messages,
    tools=functions,
    tool_choice="auto"
)

# 7.0s

In [7]:
response = response.choices[0].message
response

# response from OpenAI
# ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SPRvNtyTO6AWGqK85FMx2taW', function=Function(arguments='{"date": "2023-11-01", "time": "14:00", "attendees": ["Alice", "Bob"]}', name='schedule_meeting'), type='function')], refusal=None)
# ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pHhAr3KfObP6gPzaPNUc9oeK', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')], refusal=None)
# ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HRO0PEi7GG42M81rC5CYLRTq', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')], refusal=None)

ChatCompletionMessage(content="I'm sorry but I do not have the capability to schedule meetings currently. Nevertheless, I can help you set up a reminder or provide tips on how to manually schedule one. Would you like more information on that?", role='assistant', function_call=None, tool_calls=None)

In [8]:
print(response.tool_calls)

# response from OpenAI
# [ChatCompletionMessageToolCall(id='call_SPRvNtyTO6AWGqK85FMx2taW', function=Function(arguments='{"date": "2023-11-01", "time": "14:00", "attendees": ["Alice", "Bob"]}', name='schedule_meeting'), type='function')]
# [ChatCompletionMessageToolCall(id='call_pHhAr3KfObP6gPzaPNUc9oeK', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')]
# [ChatCompletionMessageToolCall(id='call_HRO0PEi7GG42M81rC5CYLRTq', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')]

None


In [26]:
# Check if the model wants to call our function:
if response.tool_calls:
    # Get the first function call:
    first_tool_call = response.tool_calls[0]

    # Find the function name and function args to call:
    function_name = first_tool_call.function.name
    function_args = json.loads(first_tool_call.function.arguments)
    print("This is the function name: ", function_name)
    print("These are the function arguments: ", function_args)

    function = OPENAI_FUNCTIONS.get(function_name)

    if not function:
        raise Exception(f"Function {function_name} not found.")

    # Call the function:
    function_response = function(**function_args)

    # Share the function's response with the model:
    messages.append(
        {
            "role": "function",
            "name": "schedule_meeting",
            "content": json.dumps(function_response),
        }
    )

    # Let the model generate a user-friendly response:
    # second_response = client.chat.completions.create(
    #     model="gpt-3.5-turbo-0613", messages=messages
    # )
    # second_response = client.chat.completions.create(
    #     model="gpt-3.5-turbo", messages=messages
    # )

    second_response = client.chat.completions.create(
        model=model, messages=messages
    )

    print(second_response.choices[0].message.content)


# Output from 'gpt-3.5-turbo'
# This is the function name:  schedule_meeting
# These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
# Meeting scheduled for November 1, 2023 at 14:00 with Alice and Bob. Event ID: 1234.


# This is the expected output ...

# This is the function name:  schedule_meeting
# These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
# Meeting Scheduled:

# Date: 2023-11-01
# Time: 14:00
# Attendees: Alice, Bob

# Please mark your calendars accordingly.

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}


BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'role' field that is in [user, assistant, system, tool]. Got 'function'."}

The above error message is complaining about the content of 'messages', so let's have a look at it.

In [27]:
messages

[{'role': 'user',
  'content': 'Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob.'},
 {'role': 'function',
  'name': 'schedule_meeting',
  'content': '{"event_id": "1234", "status": "Meeting scheduled successfully!", "date": "2023-11-01", "time": "14:00", "attendees": ["Alice", "Bob"]}'}]

--------------------------------

## Parallel Function Calling:

In [11]:
# Start the conversation:
messages = [
    {
        "role": "user",
        "content": '''Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob. 
        Then I want to schedule another meeting on 2023-11-02 at 15:00 with Charlie and Dave.'''
    }
]


In [12]:
# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    # model="gpt-3.5-turbo-1106",
    # model="gpt-3.5-turbo",
    model=model,
    messages=messages,
    tools=functions,
)

# 12.1 s

In [13]:
response = response.choices[0].message
response

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='382644799', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function'), ChatCompletionMessageToolCall(id='937421461', function=Function(arguments='{"date":"2023-11-02","time":"15:00","attendees":["Charlie","Dave"]}', name='schedule_meeting'), type='function')])

In [14]:
# Check if the model wants to call our function:
if response.tool_calls:
    for tool_call in response.tool_calls:
        # Get the function name and arguments to call:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        print("This is the function name: ", function_name)
        print("These are the function arguments: ", function_args)

        function = OPENAI_FUNCTIONS.get(function_name)

        if not function:
            raise Exception(f"Function {function_name} not found.")

        # Call the function:
        function_response = function(**function_args)

        # Share the function's response with the model:
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response),
            }
        )

    # Let the model generate a user-friendly response:
    # second_response = client.chat.completions.create(
    #     model="gpt-3.5-turbo-0613", messages=messages
    # )
    # second_response = client.chat.completions.create(
    #     model="gpt-3.5-turbo", messages=messages
    # )
    second_response = client.chat.completions.create(
        model=model, messages=messages
    )

    print(second_response.choices[0].message.content)

# output from 'gpt-3.5-turbo' ...
# This is the function name:  schedule_meeting
# These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
# This is the function name:  schedule_meeting
# These are the function arguments:  {'date': '2023-11-02', 'time': '15:00', 'attendees': ['Charlie', 'Dave']}
# Two meetings have been successfully scheduled:
# 1. Meeting with Alice and Bob on 2023-11-01 at 14:00
# 2. Meeting with Charlie and Dave on 2023-11-02 at 15:00.

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-02', 'time': '15:00', 'attendees': ['Charlie', 'Dave']}


BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'role' field that is in [user, assistant, system, tool]. Got 'function'."}

---

## Function Calling in LangChain

In [15]:
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional

In [16]:
class Article(BaseModel):
    """Identifying key points and contrarian views in an article."""

    points: str = Field(..., description="Key points from the article")
    contrarian_points: Optional[str] = Field(
        None, description="Any contrarian points acknowledged in the article"
    )
    author: Optional[str] = Field(None, description="Author of the article")

In [17]:
_EXTRACTION_TEMPLATE = """Extract and save the relevant entities mentioned \
in the following passage together with their properties.

If a property is not present and is not required in the function parameters, do not include it in the output."""

# Create a prompt telling the LLM to extract information
prompt = ChatPromptTemplate.from_messages(
    {("system", _EXTRACTION_TEMPLATE), ("user", "{input}")}
)

In [15]:
# model = ChatOpenAI()

In [18]:
pydantic_schemas = [Article]

# Convert Pydantic objects to the appropriate schema:
tools = [convert_to_openai_tool(p) for p in pydantic_schemas]

# Give the model access to these tools:
# model = model.bind_tools(tools=tools)
client = client.bind_tools(tools=tools)

# Create an end to end chain
# chain = prompt | model | PydanticToolsParser(tools=pydantic_schemas)
chain = prompt | client | PydanticToolsParser(tools=pydantic_schemas)

AttributeError: 'OpenAI' object has no attribute 'bind_tools'

In [17]:
result = chain.invoke(
    {
        "input": """In the recent article titled 'AI adoption in industry', key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI. However, the author, Dr. Jane Smith, acknowledges a contrarian view — that without stringent regulations, AI may pose high risks."""
    }
)
print(result)

# output from openai ...
# [Article(points='Key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI.', contrarian_points='Without stringent regulations, AI may pose high risks.', author='Dr. Jane Smith')]

[Article(points='Key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI.', contrarian_points='Without stringent regulations, AI may pose high risks.', author='Dr. Jane Smith')]


---

## Extracting Chains with LangChain

In [18]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

In [19]:
# Make sure to use a recent model that supports tools
# model = ChatOpenAI(model="gpt-3.5-turbo-1106")
model = ChatOpenAI(model="gpt-3.5-turbo")

In [20]:
class Person(BaseModel):
    """A person's name and age."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")

In [21]:
chain = create_extraction_chain_pydantic(Person, model)
chain.invoke({'input':'''Bob is 25 years old. He lives in New York. He likes to play basketball. Sarah is 30 years old. She lives in San Francisco. She likes to play tennis.'''})

# output from openai ...
# [Person(name='Bob', age=25), Person(name='Sarah', age=30)]

[Person(name='Bob', age=25), Person(name='Sarah', age=30)]